In [1]:
import numpy
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


[nltk_data] Downloading package stopwords to C:\Users\GOWTHAM-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# load data
file = open("C:\\Users\\GOWTHAM-PC\\textgen\\Pandya.txt").read()

In [17]:
# tokenization
#standardization
def tokenize_words(input):
    input = input.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)
    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return "".join(filtered)

processed_inputs = tokenize_words(file)


In [18]:
#char to num
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c,i) for i, c in enumerate(chars))

In [19]:
#check if words to char or char to num has worked
input_len = len(processed_inputs)
vocab_len = len(chars)
print("Total no of characters : ", input_len)
print("Total vocab : ", vocab_len)

Total no of characters :  455
Total vocab :  25


In [20]:
#seq length
seq_length = 100
x_data = []
y_data = []

In [21]:
# loop through the sequence
for i in range(0, input_len - seq_length):
    in_seq = processed_inputs[i:i +seq_length]
    out_seq = processed_inputs[i +seq_length]
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])
    
n_patterns = len(x_data)
print("Total patterns : ", n_patterns)

Total patterns :  355


In [22]:
# convert input sequence to np array and so on
x = numpy.reshape(x_data, (n_patterns, seq_length, 1))
x = x/float(vocab_len) 

In [23]:
#one-hot encoding
y = np_utils.to_categorical(y_data)

In [24]:
# creating sequential model
model = Sequential()
model.add(LSTM(256, input_shape=(x.shape[1], x.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))


In [25]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [26]:
# saving weights
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose = 1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [27]:
# fit model and train
model.fit(x,y, epochs=7, batch_size=256, callbacks=desired_callbacks)

Epoch 1/7
2/2 [==============================] - ETA: 0s - loss: 3.1671
Epoch 00001: loss improved from inf to 3.16714, saving model to model_weights_saved.hdf5
2/2 [==============================] - 36s 18s/step - loss: 3.1671
Epoch 2/7
2/2 [==============================] - ETA: 0s - loss: 3.0569
Epoch 00002: loss improved from 3.16714 to 3.05687, saving model to model_weights_saved.hdf5
2/2 [==============================] - 27s 13s/step - loss: 3.0569
Epoch 3/7
2/2 [==============================] - ETA: 0s - loss: 3.0016
Epoch 00003: loss improved from 3.05687 to 3.00159, saving model to model_weights_saved.hdf5
2/2 [==============================] - 23s 12s/step - loss: 3.0016
Epoch 4/7
2/2 [==============================] - ETA: 0s - loss: 2.9129
Epoch 00004: loss improved from 3.00159 to 2.91292, saving model to model_weights_saved.hdf5
2/2 [==============================] - 19s 10s/step - loss: 2.9129
Epoch 5/7
2/2 [==============================] - ETA: 0s - loss: 2.9273
Epoc

In [28]:
# recompile model with the saved weights
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [29]:
#output of the model back into characters
num_to_char = dict((i,c) for i,c in enumerate(chars))

In [30]:
# random seed to help generate
start = numpy.random.randint(0, len(x_data) -1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]),"\"")

Random Seed:
" nastysouthindiaonethreefamoustamillineagestwocholacherarulersthreetamildynastiesreferredthreecrowned "


In [31]:
#generate the text
for i in range(25):
    x = numpy.reshape(pattern,(1,len(pattern), 1))
    x = x/float(vocab_len) 
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = num_to_char[index]
    seq_in = [num_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

aaaaaaaaaaaaaaaaaaaaaaaaa